In [5]:
# Cài đặt tất cả các thư viện cần thiết với phiên bản ổn định
!pip install torch==2.1.2 
!pip install transformers==4.40.2
!pip install accelerate==0.28.0
!pip install scikit-learn==1.3.2
!pip install datasets==2.18.0
!pip install bitsandbytes==0.42.0
!pip install sentencepiece==0.1.99

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

# Điền access token của bạn vào đây
login(token="YOUR_HUGGINGFACE_TOKEN")


/home/team_voice/miniconda3/envs/duration/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "Viet-Mistral/Vistral-7B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

chat_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=64)


/home/team_voice/miniconda3/envs/duration/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.54s/it]


In [3]:
import json

# with open('/SSD/team_voice/vmnghia/workdir/VLSP/duration_training_dataset.txt', 'r', encoding='utf-8') as f:
#     data = [json.loads(line.strip()) for line in f if line.strip()]

with open('/SSD/team_voice/vmnghia/workdir/VLSP/vistral_predictions.txt', 'r', encoding='utf-8') as f:
    data = [json.loads(line.strip()) for line in f if line.strip()]


In [16]:
def classify_option(context, question, option):
    prompt = f"""
Bạn hãy trả lời "yes" hoặc "no" cho lựa chọn thời gian dưới đây có phù hợp với ngữ cảnh hay không.

Ngữ cảnh: {context}
Câu hỏi: {question}
Lựa chọn: {option}

Trả lời chỉ bằng "yes" hoặc "no":
"""
    output = chat_pipeline(prompt)[0]["generated_text"]
    for line in output.split('\n')[::-1]:
        if "yes" in line.lower():
            return "yes"
        if "no" in line.lower():
            return "no"
    return "no"


In [17]:
from tqdm import tqdm

output_data = []

for ex in tqdm(data):  # dùng [:20] để test nhanh, bỏ giới hạn để chạy toàn bộ
    preds = []
    for opt in ex["options"]:
        pred = classify_option(ex["context"], ex["question"], opt)
        preds.append(pred)

    # Ghi full thông tin + dự đoán
    output_data.append({
        "context": ex["context"],
        "question": ex["question"],
        "options": ex["options"],
        "qid": ex["qid"],
        "labels": ex["labels"],
        "predictions": preds
    })

# Ghi từng dòng JSONL
with open("/SSD/team_voice/vmnghia/workdir/VLSP/vistral_predictions.txt", "w", encoding="utf-8") as f:
    for item in output_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")


  9%|▊         | 128/1490 [01:48<19:15,  1.18it/s]


KeyboardInterrupt: 

In [4]:
# Load lại file output đã ghi
with open("/SSD/team_voice/vmnghia/workdir/VLSP/vistral_predictions.txt", "r", encoding="utf-8") as f:
    pred_data = [json.loads(line) for line in f]

# Tính toán các chỉ số đánh giá
from sklearn.metrics import precision_score, recall_score, f1_score

def compute_metrics(y_true, y_pred):
    exact_match = int(y_true == y_pred)
    y_true_bin = [1 if x == "yes" else 0 for x in y_true]
    y_pred_bin = [1 if x == "yes" else 0 for x in y_pred]
    p = precision_score(y_true_bin, y_pred_bin, zero_division=0)
    r = recall_score(y_true_bin, y_pred_bin, zero_division=0)
    f1 = f1_score(y_true_bin, y_pred_bin, zero_division=0)
    return exact_match, p, r, f1

# Đánh giá tổng thể
total_em, total_p, total_r, total_f1 = 0, 0, 0, 0
n = len(pred_data)

for ex in pred_data:
    em, p, r, f1 = compute_metrics(ex["labels"], ex["predictions"])
    total_em += em
    total_p += p
    total_r += r
    total_f1 += f1

print(f"Exact Match: {total_em/n:.4f}")
print(f"Precision:    {total_p/n:.4f}")
print(f"Recall:       {total_r/n:.4f}")
print(f"F1-score:     {total_f1/n:.4f}")


Exact Match: 0.1154
Precision:    0.5633
Recall:       0.9006
F1-score:     0.6736
